In [3]:
import scipy.linalg
import scipy.optimize
import sys
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_function as tfu
tfu = importlib.reload(tfu)
from scipy.interpolate import splprep, splev
from scipy import interpolate
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from scipy.spatial.transform import Rotation as R
import scripts.theodolite_utils as ttfu
import time
from tqdm import tqdm
import scripts.gp_prediction_utils as GPf
GPf = importlib.reload(GPf)

ModuleNotFoundError: No module named 'GPy'

In [2]:
def save_results_drop_outliers(file_name_path, param, results_arr):
    file_name = file_name_path+str(param[0])+"-"+str(param[1])+"-"+str(param[2])+"-"+str(param[3])+"-"+str(param[4])+"-"+str(param[5])+".txt"
    file = open(file_name,"w+")
    file.write(str(results_arr[0]))
    file.write(" ")
    file.write(str(results_arr[1]))
    file.write(" ")
    file.write(str(results_arr[2]))
    file.write(" ")
    file.write(str(results_arr[3]))
    file.write(" ")
    file.write(str(results_arr[4]))
    file.write(" ")
    file.write(str(results_arr[5]))
    file.write("\n")
    file.close()
    
def read_results_drop_outliers(file_name):
    data = []
    with open(file_name, "r") as file:
        for line in file:
            item = line.strip().split(" ")
            data.append(item)
    return data
    
def drop_points_filtering(file_name_path, param, repo_out):
    number_points_input = []
    number_points_out_outliers = []
    number_points_out_filters_only = []
    number_points_out_filters = []
    number_points_out_outliers_end_with_outliers = []
    number_points_out_outliers_end_without_ouliers = []
    
    for i, repo_out_name in zip(file_name_path,repo_out):
        print(i)
        
        # Data filtered with outliers
        t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttfu.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(i)
        index_1_f = ttfu.thresold_raw_data(t1, d1, a1, e1, param[0], param[1]*3.1415926/180, param[2]*3.1415926/180, param[3])
        index_2_f = ttfu.thresold_raw_data(t2, d2, a2, e2, param[0], param[1]*3.1415926/180, param[2]*3.1415926/180, param[3])
        index_3_f = ttfu.thresold_raw_data(t3, d3, a3, e3, param[0], param[1]*3.1415926/180, param[2]*3.1415926/180, param[3])
        t1 = t1[index_1_f]
        t2 = t2[index_2_f]
        t3 = t3[index_3_f]
        tp1 = tp1[index_1_f].T
        tp2 = tp2[index_2_f].T
        tp3 = tp3[index_3_f].T

        # Raw data
        tr1, tr2, tr3, trp1, trp2, trp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttfu.read_rosbag_theodolite_without_tf_raw_data(i)

        sum_input_points = len(tr1)+len(tr2)+len(tr3)
        sum_outliers_points = len(t1)+len(t2)+len(t3)
        
        # data filtered in pipeline
        list_interval, list_time = tfu.split_time_interval_all_data(t1, t2, t3, param[3])
        list_trajectories_split = tfu.merge_interval(list_interval, list_time, t1, t2, t3, param[3])
        
        sum_filters_points = 0
        sum_filters_points_with_outliers = 0
        
        for j in tqdm(list_trajectories_split):
            index_1 = np.array([j[0,0],j[1,0]])
            index_2 = np.array([j[0,1],j[1,1]])
            index_3 = np.array([j[0,2],j[1,2]])
            
            sum_filters_points = sum_filters_points + len(t1[index_1[0]:index_1[1]]) + len(t2[index_2[0]:index_2[1]]) + len(t3[index_3[0]:index_3[1]])
            
            index_1 = GPf.delta_t_function(index_1,t1,param[4])
            index_2 = GPf.delta_t_function(index_2,t2,param[4])
            index_3 = GPf.delta_t_function(index_3,t3,param[4])
                
            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if(abs(end-begin)>param[5] and begin<end):
                sum_filters_points_with_outliers = sum_filters_points_with_outliers + len(t1[index_1[0]:index_1[1]]) + len(t2[index_2[0]:index_2[1]]) + len(t3[index_3[0]:index_3[1]]) 
            
        # Raw data in pipeline
        list_interval, list_time = tfu.split_time_interval_all_data(tr1, tr2, tr3, param[3])
        list_trajectories_split = tfu.merge_interval(list_interval, list_time, tr1, tr2, tr3, param[3])
        
        sum_filters_points_only = 0
        sum_filters_points_without_outliers = 0
        
        for j in tqdm(list_trajectories_split):
            index_1 = np.array([j[0,0],j[1,0]])
            index_2 = np.array([j[0,1],j[1,1]])
            index_3 = np.array([j[0,2],j[1,2]])
            
            sum_filters_points_only = sum_filters_points_only + len(tr1[index_1[0]:index_1[1]]) + len(tr2[index_2[0]:index_2[1]]) + len(tr3[index_3[0]:index_3[1]])

            index_1 = GPf.delta_t_function(index_1,tr1,param[4])
            index_2 = GPf.delta_t_function(index_2,tr2,param[4])
            index_3 = GPf.delta_t_function(index_3,tr3,param[4])
                
            begin = np.max([tr1[index_1[0]], tr2[index_2[0]], tr3[index_3[0]]])
            end = np.min([tr1[index_1[1]], tr2[index_2[1]], tr3[index_3[1]]])

            if(abs(end-begin)>param[5] and begin<end):
                sum_filters_points_without_outliers = sum_filters_points_without_outliers + len(tr1[index_1[0]:index_1[1]]) + len(tr2[index_2[0]:index_2[1]]) + len(tr3[index_3[0]:index_3[1]]) 

        number_points_input.append(sum_input_points)
        number_points_out_outliers.append(sum_outliers_points)
        number_points_out_filters.append(sum_filters_points)
        number_points_out_filters_only.append(sum_filters_points_only)
        number_points_out_outliers_end_with_outliers.append(sum_filters_points_with_outliers)
        number_points_out_outliers_end_without_ouliers.append(sum_filters_points_without_outliers)
        
        results_arr = np.array([sum_input_points, sum_outliers_points, sum_filters_points, sum_filters_points_only, sum_filters_points_with_outliers, sum_filters_points_without_outliers])
        save_results_drop_outliers(repo_out_name, param, results_arr)

    print("Results done !")
    
    return number_points_input, number_points_out_outliers, number_points_out_filters, number_points_out_filters_only, number_points_out_outliers_end_with_outliers, number_points_out_outliers_end_without_ouliers

In [ ]:
file = ["/home/norlab/Data/IROS_2022/20220224_TS/2022-02-24-15-34-38.bag",
        "/home/norlab/Data/IROS_2022/20220307_TS/2022-03-07-19-20-06.bag",
        "/home/norlab/Data/IROS_2022/20220312_TS/2022-03-12-09-45-12.bag",
        "/home/norlab/Data/IROS_2022/20220314_TS/2022-03-14-10-47-49.bag",
        "/home/norlab/Data/IROS_2022/20220316_TS/2022-03-16-19-02-42.bag",
        "/home/norlab/Data/IROS_2022/20220331_TS/2022-03-31-10-22-52.bag",
        "/home/norlab/Data/IROS_2022/20220331_TS/2022-03-31-11-20-05.bag",
        "/home/norlab/Data/IROS_2022/20220427_TS/2022-04-27-12-12-10_filered.bag",
        "/home/norlab/Data/IROS_2022/20220505_TS/empty1_2022-05-05-19-14-33.bag",
        "/home/norlab/Data/IROS_2022/20220505_TS/cones1_2022-05-05-19-25-54.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-10-30-37_filtered.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-11-01-56.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-11-24-28.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-21-49-12_filtered.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-22-35-30.bag",
        "/home/norlab/Data/IROS_2022/20220513_TS/2022-05-13-22-51-40.bag",
        "/home/norlab/Data/IROS_2022/20220523_TS/cones_filtered_2022-05-23-19-31-32.bag",
        "/home/norlab/Data/IROS_2022/20220523_TS/constrained_2022-05-23-18-56-16.bag",
        "/home/norlab/Data/IROS_2022/20220523_TS/empty_filtered_2022-05-23-19-18-35.bag",
        "/home/norlab/Data/IROS_2022/20220523_TS/other_tunnel_2022-05-23-19-39-40.bag",
        "/home/norlab/Data/IROS_2022/20220525_TS/2022-05-25-11-07-40.bag",
        "/home/norlab/Data/IROS_2022/20220525_TS/2022-05-25-13-32-32.bag",
        "/home/norlab/Data/IROS_2022/20220622_TS/2022-06-22-17-48-25.bag",
        "/home/norlab/Data/IROS_2022/20220622_TS/2022-06-22-18-11-00.bag",
        "/home/norlab/Data/IROS_2022/20220630_TS/2022-06-30-10-32-21.bag",
        "/home/norlab/Data/IROS_2022/20220630_TS/2022-06-30-10-57-07.bag",
        "/home/norlab/Data/IROS_2022/20220711_TS/2022-07-11-15-36-04.bag",
        "/home/norlab/Data/IROS_2022/20220711_TS/2022-07-11-15-50-00.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-11-50-15.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-11-53-12.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-11-58-58.bag",
        "/home/norlab/Data/IROS_2022/20220715_TS/2022-07-15-14-42-27.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-10-31.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-11-10.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/cones_2022-07-17-20-38-29.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/empty_2022-07-17-21-03-14.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/calibration_2022-07-17-21-27-28.bag"
       ]

repo_out = [
    './data/drop_outliers/20220224/',
    './data/drop_outliers/20220307/',
    './data/drop_outliers/20220312/',
    './data/drop_outliers/20220314/',
    './data/drop_outliers/20220316/',
    './data/drop_outliers/20220331-1/',
    './data/drop_outliers/20220331-2/',
    './data/drop_outliers/20220427/',
    './data/drop_outliers/20220505_empty/',
    './data/drop_outliers/20220505_cones/',
    './data/drop_outliers/20220513/01/',
    './data/drop_outliers/20220513/02/',
    './data/drop_outliers/20220513/03/',
    './data/drop_outliers/20220513/04/',
    './data/drop_outliers/20220513/05/',
    './data/drop_outliers/20220513/06/',
    './data/drop_outliers/20220523_cones/',
    './data/drop_outliers/20220523_constrained/',
    './data/drop_outliers/20220523_empty/',
    './data/drop_outliers/20220523_other_tunnel/',
    './data/drop_outliers/20220525/01/',
    './data/drop_outliers/20220525/02/',
    './data/drop_outliers/20220622-1/',
    './data/drop_outliers/20220622-2/',
    './data/drop_outliers/20220630-1/',
    './data/drop_outliers/20220630-2/',
    './data/drop_outliers/20220711/01/',
    './data/drop_outliers/20220711/02/',
    './data/drop_outliers/20220715/01/',
    './data/drop_outliers/20220715/02/',
    './data/drop_outliers/20220715/03/',
    './data/drop_outliers/20220715/04/',
    './data/drop_outliers/20220717/01/',
    './data/drop_outliers/20220717/02/',
    './data/drop_outliers/20220717/03/',
    './data/drop_outliers/20220717/04/',
    './data/drop_outliers/20220717/05/'
]

# Outlier range
#param = [np.array([0.5, 10, 10, 1, 0, 6]),
#         np.array([1, 10, 10, 1, 0, 6]),
#         np.array([1.5, 10, 10, 1, 0, 6]),
#         np.array([2, 10, 10, 1, 0, 6]),
#         np.array([2.5, 10, 10, 1, 0, 6]),
#         np.array([3, 10, 10, 1, 0, 6]),
#        ]

# Outlier azimuth angle
#param = [np.array([10, 0.5, 10, 1, 0, 6]),
#         np.array([10, 1, 10, 1, 0, 6]),
#         np.array([10, 1.5, 10, 1, 0, 6]),
#         np.array([10, 2, 10, 1, 0, 6]),
#         np.array([10, 2.5, 10, 1, 0, 6]),
#         np.array([10, 3, 10, 1, 0, 6]),
#        ]

# Outlier elevation angle
#param = [np.array([10, 10, 0.5, 1, 0, 6]),
#         np.array([10, 10, 1, 1, 0, 6]),
#         np.array([10, 10, 1.5, 1, 0, 6]),
#         np.array([10, 10, 2, 1, 0, 6]),
#         np.array([10, 10, 2.5, 1, 0, 6]),
#         np.array([10, 10, 3, 1, 0, 6]),
#        ]

# split time interval 
#param = [np.array([2, 1, 1, 0.5, 0, 6]),
#         np.array([2, 1, 1, 0.75, 0, 6]),
#         np.array([2, 1, 1, 1, 0, 6]),
#         np.array([2, 1, 1, 1.25, 0, 6]),
#         np.array([2, 1, 1, 1.5, 0, 6]),
#         np.array([2, 1, 1, 1.75, 0, 6]),
#         np.array([2, 1, 1, 2, 0, 6]),
#         np.array([2, 1, 1, 2.25, 0, 6]),
#         np.array([2, 1, 1, 2.5, 0, 6])
#        ]

# delta_t 
param = [np.array([2, 1, 1, 1, 0, 6]),
         np.array([2, 1, 1, 1, 0.25, 6]),
         np.array([2, 1, 1, 1, 0.5, 6]),
         np.array([2, 1, 1, 1, 0.75, 6]),
         np.array([2, 1, 1, 1, 1, 6]),
         np.array([2, 1, 1, 1, 1.25, 6]),
         np.array([2, 1, 1, 1, 1.5, 6]),
         np.array([2, 1, 1, 1, 1.75, 6]),
         np.array([2, 1, 1, 1, 2, 6])
        ]

# split time interval 
#param = [np.array([2, 1, 1, 1, 0, 2]),
#         np.array([2, 1, 1, 1, 0, 4]),
#         np.array([2, 1, 1, 1, 0, 6]),
#         np.array([2, 1, 1, 1, 0, 8]),
#         np.array([2, 1, 1, 1, 0, 10]),
#         np.array([2, 1, 1, 1, 0, 12])
#        ]

In [ ]:
mean_points_input = []
mean_point_out_outlier = []
mean_point_out_filter = []
mean_point_out_filter_only = []
mean_points_out_outliers_end_with_outliers = []
mean_points_out_outliers_end_without_outliers = []
for i in param:
    print("**********************************************")
    print("**********************************************")
    print(i)
    number_points_input, number_points_out_outliers, number_points_out_filters, number_points_out_filters_only, number_points_out_outliers_end_with_outliers, number_points_out_outliers_end_without_ouliers = drop_points_filtering(file, i, repo_out)
    mean_points_input.append(np.sum(number_points_input))
    mean_point_out_outlier.append(np.sum(number_points_out_outliers))
    mean_point_out_filter.append(np.sum(number_points_out_filters))
    mean_point_out_filter_only.append(np.sum(number_points_out_filters_only))
    mean_points_out_outliers_end_with_outliers.append(np.sum(number_points_out_outliers_end_with_outliers))
    mean_points_out_outliers_end_without_outliers.append(np.sum(number_points_out_outliers_end_without_ouliers))
    

In [ ]:
print(param[0])
file_name = repo_out[0]+str(param[0][0])+"-"+str(param[0][1])+"-"+str(param[0][2])+"-"+str(param[0][3])+"-"+str(param[0][4])+"-"+str(param[0][5])+".txt"
results = read_results_drop_outliers(file_name)
print(results)


In [ ]:
# Percentage points removed by outliers/filters

percentage_out_outliers = []
percentage_out_filters = []
percentage_out_total_outliers_filters = []
percentage_out_end_outliers_only = []
percentage_out_total = []
for i,j,k,l,m,n in zip(mean_points_input, mean_point_out_outlier, mean_point_out_filter, mean_point_out_filter_only, mean_points_out_outliers_end_with_outliers, mean_points_out_outliers_end_without_outliers):
    percentage_outliers_only = (i-j)/i*100
    percentage_filters_only = (i-l)/i*100
    percentage_total_outliers_filters = (i-k)/i*100
    percentage_end_outliers_only = abs(l-n)/i*100
    percentage_total = (i-m)/i*100
    
    percentage_out_outliers.append(percentage_outliers_only)
    percentage_out_filters.append(percentage_filters_only)
    percentage_out_total_outliers_filters.append(percentage_total_outliers_filters)
    percentage_out_end_outliers_only.append(percentage_end_outliers_only)
    percentage_out_total.append(percentage_total)
    

In [ ]:
%matplotlib notebook

y_limit = 360000
#arr = ["0.5", "1", "1.5", "2", "2.5", "3"]  #30min for distance outliers generation, azimuth, elevation
#arr = ["0", "0.25","0.5","0.75", "1", "1.25","1.5","1.75", "2"]  #50min for delta_t outliers generation
#arr = ["0.5", "0.75","1","1.25", "1.5", "1.75","2","2.25", "2.5"]  #50min for time split interval generation
arr = ["2", "4","6","8", "10", "12"]  #40min for length interval generation

#arr = ["0.5","1","1.5","2"]
#arr = ["0.5","1","1.5","2"]

fig = plt.figure(figsize =(8,8))
ax = fig.add_subplot(211)
ax.plot(mean_points_input, color='r', label="total points")
ax.plot(mean_point_out_outlier, color='b', label="points outlier")
ax.plot(mean_point_out_filter, color='g', label="points filter")
ax.plot(mean_points_out_outliers_end_with_outliers, color='yellow', label="points end outliers")
ax.set_ylabel("Number points")
ax.set_ylim([0,y_limit])
ax.legend(mode = "expand", ncol = 4)
#plt.xticks([0, 1, 2, 3, 4, 5], arr)
#ax.set_xlabel("Outlier distance [m.s^-1]")
#ax.set_xlabel("Outlier azimuth [deg/s]")
#ax.set_xlabel("Outlier elevation [deg/s]")
#plt.xticks([0, 1, 2, 3, 4, 5,6,7], arr)
#ax.set_xlabel("Outlier delta_t [s]")
#ax.set_xlabel("Time split [s]")
plt.xticks([0, 1, 2, 3, 4, 5], arr)
ax.set_xlabel("Length interval [s]")

ax2 = fig.add_subplot(212)
ax2.plot(percentage_out_outliers, color='b', label="outlier")
ax2.plot(percentage_out_filters, color='g', label="filtered")
ax2.plot(percentage_out_end_outliers_only, color='r', label="end outliers")
ax2.plot(percentage_out_total, color='black', label="total")
ax2.legend(mode = "expand", ncol = 4)
ax2.set_ylim([0,100])
ax2.set_ylabel("Percentage removed [%]")
#plt.xticks([0, 1, 2, 3, 4, 5], arr)
#ax2.set_xlabel("Outlier distance [m.s^-1]")
#ax2.set_xlabel("Outlier azimuth [deg/s]")
#ax2.set_xlabel("Outlier elevation [deg/s]")
#plt.xticks([0, 1, 2, 3, 4, 5,6,7], arr)
#ax2.set_xlabel("Outlier delta_t [s]")
#ax2.set_xlabel("Time split [s]")
plt.xticks([0, 1, 2, 3, 4, 5], arr)
ax2.set_xlabel("Length interval [s]")

fig.tight_layout()
plt.show()
#fig.savefig("./figs/number_points_removed_distance-f-n-10-10-1-0-6.jpg")
#fig.savefig("./figs/number_points_removed_azimuth-f-10-n-10-1-0-6.jpg")
#fig.savefig("./figs/number_points_removed_elevation-f-10-10-n-1-0-6.jpg")
#fig.savefig("./figs/number_points_removed_delta_t-f-2-1-1-1-n-6.jpg")
#fig.savefig("./figs/number_points_removed_time_split-f-2-1-1-n-0-6.jpg")
#fig.savefig("./figs/number_points_removed_length_interval-f-2-1-1-1-0-n.jpg")